# Tests



In [1]:
from openmm import app
from openmm import *
from openmm import unit as u
import molsysmt as msm
from sys import stdout

In [2]:
pdb = app.PDBFile('V5.pdb')

In [3]:
modeller = app.Modeller(pdb.topology, pdb.positions)

In [4]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [5]:
modeller.addSolvent(forcefield, model='tip3p', padding=1.4*u.nanometers, boxShape='cube')

In [6]:
aa = list(modeller.topology.atoms())

In [11]:
aa[-1].element.symbol='CL'

AttributeError: property 'symbol' of 'Element' object has no setter

In [ ]:
app.PDBFile.writeFile(modeller.topology, modeller.positions, file="V5_solvated.pdb")

In [ ]:
msm.view(modeller, standard=True, with_water_as='licorice', with_ions_as='balls')

In [ ]:
msm.info(modeller)

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.2*u.nanometer, constraints=app.HBonds)

In [ ]:
friction = 1 / u.picosecond
timestep = 2 * u.femtoseconds
temperature = 500 * u.kelvin
integrator = LangevinIntegrator(temperature, friction, timestep)

In [ ]:
platform = Platform.getPlatformByName("CUDA")  # Usa "CPU" si no tienes GPU

In [ ]:
simulation = app.Simulation(modeller.topology, system, integrator, platform)

In [ ]:
simulation.context.setPositions(modeller.positions)

In [ ]:
simulation.minimizeEnergy()

In [ ]:
reporter1 = app.DCDReporter('traj.dcd', 2000)
reporter2 = app.StateDataReporter(stdout, 10000, step=True, potentialEnergy=True, temperature=True)

In [ ]:
simulation.reporters = [reporter1, reporter2]

In [ ]:
simulation.step(50000)

In [ ]:
molsys = msm.convert(['V5_solvated.pdb','traj.dcd'])

In [ ]:
msm.info(molsys)

In [ ]:
msm.view(molsys, standard=True)

In [ ]:
msm.element.entity.get_entity_index(pdb.topology, element='atom', selection='all', redefine_molecules=True,
                                    redefine_indices=True, skip_digestion=True)

In [ ]:
msm.get(pdb.topology, element='atom', entity_index=True)

In [ ]:
msm.info(pdb.topology)